In [1]:
import os
from os import mkdir
from os.path import isdir, join
from shutil import copyfile
import glob
import numpy as np

os.environ['NUMEXPR_MAX_THREADS'] = '16'

In [9]:
files_dir = '../../data/kitti/raw/dataset/sequences/00/velodyne'
files = sorted(glob.glob(join(files_dir, '*.bin'))) # KITTI point clouds

poses = np.loadtxt('../../data/kitti/raw/dataset/refined_poses/00.txt') # refined poses (transformations from camera frame to local frame)

Tr = np.array([[4.276802385584e-04, -9.999672484946e-01, -8.084491683471e-03, -1.198459927713e-02],
               [-7.210626507497e-03, 8.081198471645e-03, -9.999413164504e-01, -5.403984729748e-02],
               [9.999738645903e-01, 4.859485810390e-04, -7.206933692422e-03, -2.921968648686e-01],
               [0, 0, 0, 1]]) # transformation from lidar frame to camera frame

# Ground truth of rover, input
# gt = np.array([[-4131763.68069097, 2895922.7736556, -3888756.58216884]]).T
# gt = np.array([[-4131574.26909343, 2897161.19590152, -3887951.67177844]]).T # PKVL, computed by reference velocity, ITRF2014@04/05/2022

# transformation from [0, 0, 0] to target point
# T_rov_target = np.hstack((np.eye(3), gt))
# T_rov_target = np.vstack((T_rov_target, np.array([[0, 0, 0, 1]])))

spacer = 10 # number of seconds between rov and ref, input
epochs = range(324000, 324000+3600) # epochs in GPS TOW, input

# output directory
output_dir = '../../../KITTI_partial_' + str(spacer) + 's' + '_220504'
if not isdir(output_dir):
  mkdir(output_dir)

rov_dir = join(output_dir, 'velodyne_rov') # rov scan directory
if not isdir(rov_dir):
  mkdir(rov_dir)

ref_dir = join(output_dir, 'velodyne_ref') # ref scan directory
if not isdir(ref_dir):
  mkdir(ref_dir)

In [10]:
T_ref_rov_all = np.empty([len(epochs)*4, 4]) # transformations from ref to rov per epoch
# T_ref_target_all = np.empty([len(epochs)*4, 4]) # transformations from ref to target per epoch
test_list = np.empty([len(epochs), 1]) # rover scan epochs

for i in range(len(epochs)):
  rov_id = epochs[i] # TOW of rover scan
  ref_id = rov_id + 9000000
  
  rov_file = files[i]
  ref_file = files[i+spacer]

  # Copy and rename rov and ref scans
  copyfile(rov_file, join(rov_dir, str(rov_id) + '.bin'))
  copyfile(ref_file, join(ref_dir, str(ref_id) + '.bin'))

  # Get transformation from ref to rov
  T_rov_local = np.concatenate((poses[i, :].reshape((3, 4)), np.array([[0, 0, 0, 1]])), axis=0)
  T_rov_local = np.dot(np.dot(np.linalg.inv(Tr), T_rov_local), Tr)

  T_ref_local = np.concatenate((poses[i+spacer, :].reshape((3, 4)), np.array([[0, 0, 0, 1]])), axis=0)
  T_ref_local = np.dot(np.dot(np.linalg.inv(Tr), T_ref_local), Tr)

  T_ref_rov = np.dot(np.linalg.inv(T_rov_local), T_ref_local) # ref lidar frame to rov lidar frame
  T_ref_rov_all[i*4:(i+1)*4, :] = T_ref_rov

  # T_ref_target = np.dot(T_rov_target, T_ref_rov) # ref lidar frame to target
  # T_ref_target_all[i*4:(i+1)*4, :] = T_ref_target

  test_list[i, :] = rov_id

np.savetxt(join(output_dir, 'T_ref_rov.csv'), T_ref_rov_all, delimiter=',')
# np.savetxt(join(output_dir, 'Tr_georeferencing_sim.csv'), T_ref_target_all, delimiter=',')
np.savetxt(join(output_dir, 'test_list.txt'), test_list, fmt='%d', delimiter=',')